In [1]:
import tweepy
import pandas as pd
import time
import os
from datetime import datetime
from secret import consumer_key, consumer_secret, access_key, access_secret

### Twitter API authorization

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

### Scrape tweets

In [3]:
def scrape_tweets(hashtags, date_since, num_tweets, num_runs):
    
    # Dataframe to store the tweets
    tweets_df = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags'])
    # Make filename to save under
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_vegan_tweets.csv'
    
    program_start = time.time()
    for i in range(1, num_runs):
        print('run:',i)
        start_run = time.time()

        tweets = tweepy.Cursor(api.search, q=hashtags, lang="en", since=date_since,
                               tweet_mode='extended').items(num_tweets)
        tweet_list = [tweet for tweet in tweets]

        for tweet in tweet_list:
        # Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']
            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                text = tweet.full_text
            # Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]
            # Append to dataframe - db_tweets
            tweets_df.loc[len(tweets_df)] = ith_tweet

            
        print('number of tweets to save so far: ', len(tweets_df))
        tweets_df.to_csv(filename, index = False)
        print('Saved tweets as', filename)
        
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
        time.sleep(920) #15 minute sleep time
    
    program_end = time.time()
    print('Scraping completed')
    print('Total time taken to scrape is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [8]:
def scrape_tweets(hashtags, date_since, num_tweets, num_runs):
    
    # Dataframe to store the tweets
    tweets_df = pd.DataFrame(columns = ['username', 'tweetcreatedts', 'text', 'hashtags'])
    
    # Make filename to save df under
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_vegan_tweets.csv'
    
    program_start = time.time()
    for i in range(1, num_runs):
        print('run:',i)
        start_run = time.time()

        tweets = tweepy.Cursor(api.search, q=hashtags, lang="en", since=date_since,
                               tweet_mode='extended').items(num_tweets)
        tweet_list = [tweet for tweet in tweets]

        for tweet in tweet_list:
            username = tweet.user.screen_name
            tweetcreatedts = tweet.created_at
            hashtags = tweet.entities['hashtags']
            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:
                text = tweet.full_text

            tweet_data = [username, tweetcreatedts, text, hashtags]
            # Append to dataframe - db_tweets
            tweets_df.loc[len(tweets_df)] = tweet_data

            
        print('number of tweets to save so far: ', len(tweets_df))
        tweets_df.to_csv(filename, index = False)
        print('Saved tweets as', filename)
        
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
        time.sleep(920) #15 minute sleep time
    
    program_end = time.time()
    print('Scraping completed')
    print('Total time taken to scrape: {} minutes.'.format(round(program_end - program_start)/60, 2))

In [9]:
vegan_hashtags = '#veganfortheanimals OR #vegancommunity OR #veganpower OR #veganbeauty OR #vegan OR #veggie OR #plantbased OR #veganfood OR #crueltyfree OR #govegan OR #veganlife OR #veganism'
date_since = "2020-04-30"

scrape_tweets(vegan_hashtags, date_since, 2500, 6)

run: 1


TweepError: Twitter error response: status code = 429